#Copy-and-Paste Set-up Code for Pyomo w/ Solvers


In [1]:
# Install pyomo
%pip install pyomo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

#%%capture
#import sys
#import os

# If this is being run in VS Code, install the necessary packages
#if 'vscode' in os.environ.get('TERM_PROGRAM', ''):
#    !pip install pyomo
#    !pip install idaes-pse --pre
#    !python -m idaes get-extensions --to ./bin
#    !setx PATH "%PATH%;%cd%\bin"

# If this is being run in Google Colab, install the necessary packages
#if 'google.colab' in sys.modules:
#    !pip install idaes-pse --pre
#    !idaes get-extensions --to ./bin
#    os.environ['PATH'] += ':bin'

#from pyomo.environ import *
#SolverFactory('ipopt').available()
#SolverFactory('cbc').available()

# Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
# for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

# %%capture
import sys
import os
import platform

# Where we'll drop solver binaries
BIN_DIR = os.path.abspath("./bin")

# If this is being run in VS Code, install the necessary packages
# (TERM_PROGRAM contains "vscode" in VS Code terminals; in VS Code Jupyter we also have JPY_PARENT_PID)
if ('vscode' in os.environ.get('TERM_PROGRAM', '').lower()) or ('JPY_PARENT_PID' in os.environ):
    # Use pip via the current interpreter for reliability
    !{sys.executable} -m pip install -q pyomo
    !{sys.executable} -m pip install -q idaes-pse --pre
    # Use python -m idaes so it runs in the same env
    !{sys.executable} -m idaes get-extensions --to "{BIN_DIR}"
    # Make solvers visible **now** in this kernel/session
    os.environ['PATH'] = BIN_DIR + os.pathsep + os.environ.get('PATH', '')

# If this is being run in Google Colab, install the necessary packages
if 'google.colab' in sys.modules:
    !pip install -q idaes-pse --pre
    !idaes get-extensions --to "{BIN_DIR}"
    # Make solvers visible **now** in this kernel/session
    os.environ['PATH'] = BIN_DIR + os.pathsep + os.environ.get('PATH', '')

from pyomo.environ import *

# Helper to build the solver executable path, cross-platform
def _solver_exec(name: str) -> str:
    exe = name + (".exe" if platform.system().lower().startswith("win") else "")
    path = os.path.join(BIN_DIR, exe)
    return path if os.path.exists(path) else name  # fall back to PATH if needed

# Show availability
print("ipopt available? ", SolverFactory('ipopt', executable=_solver_exec('ipopt')).available(False))
print("cbc   available? ", SolverFactory('cbc',   executable=_solver_exec('cbc')).available(False))


ipopt available?  True
cbc   available?  True


# Testing and Solving with Different Solvers

This notebook contains codes for installing Pyomo and various open-source solvers that we'll leverage, including:

* COIN-OR CBC
* COIN-OR Ipopt
* COIN-OR Bonmin
* COIN-OR Couenne

It then demonstrates the use of various solvers with an example.

You need not memorize or understand the code. Simply copy and paste the relevant pieces when needed.

## Testing Pyomo

The installation of pyomo can be verified by entering a simple model. This model is used in subsequent cells to demonstrate the installation and operation of the solvers.

In [11]:
# create a model
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(expr = 40*model.x + 30*model.y, sense=maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

model.pprint()

def display_solution(model):

    # display solution
    print('\nProfit = ', model.profit())

    print('\nDecision Variables')
    print('x = ', model.x.value)
    print('y = ', model.y())

    print('\nConstraints')
    print('Demand  = ', model.demand())
    print('Labor A = ', model.laborA())
    print('Labor B = ', model.laborB())

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y : 100.0 :   True

6 Declarations: x y profit demand laborA laborB


## COIN-OR CBC

[COIN-OR CBC](https://github.com/coin-or/Cbc) is a multi-threaded open-source **C**oin-or **b**ranch and **c**ut mixed-integer linear programming solver written in C++ under the Eclipse Public License (EPL). CBC is generally a good choice for a general purpose MILP solver for medium to large scale problems.

In [12]:
SolverFactory('cbc').solve(model).write()

# display solution
display_solution(model)

cbc


ApplicationError: No executable found for solver 'cbc'

## COIN-OR Ipopt

[COIN-OR Ipopt](https://github.com/coin-or/Ipopt) is an open-source **I**nterior **P**oint **Opt**imizer for large-scale nonlinear optimization available under the Eclipse Public License (EPL). It is well-suited to solving nonlinear programming problems without integer or binary constraints.

In [3]:
SolverFactory('ipopt').solve(model).write()

# display solution
display_solution(model)

ipopt


ApplicationError: No executable found for solver 'ipopt'

## COIN-OR Bonmin

[COIN-OR Bonmin](https://www.coin-or.org/Bonmin/Intro.html) is a **b**asic **o**pen-source solver for **n**onlinear **m**ixed-**in**teger programming problems (MINLP). It utilizes CBC and Ipopt for solving relaxed subproblems.

In [ ]:
SolverFactory('bonmin').solve(model).write()

# display solution
display_solution(model)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 0.016574859619140625
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit =  2600.000025999979

## COIN-OR Couenne

COIN-OR Couenne](https://www.coin-or.org/Couenne/)  is attempts to find global optima for mixed-integer nonlinear programming problems (MINLP).

In [ ]:
SolverFactory('couenne').solve(model).write()

# display solution
display_solution(model)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: couenne\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 0.01709890365600586
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit =  2600.00002599998
